# 1. Data processing code

In [ ]:
import os
import shutil
import pandas as pd
import pyarrow.parquet as pq
from pyarrow import fs
import pyarrow as pa

In [ ]:
def combine_parquet(source_folder, output_file):
    """
    Combine Parquet files from a nested folder structure into a single Parquet file.
    The files are identified based on the folder name containing 'id='.

    Args:
        source_folder (str): Path to the folder containing subfolders with Parquet files.
        output_file (str): Path where the combined Parquet file will be saved.
    """
        # List of columns to be written to the combined file
    columns = ['id', 'relative_date_PCIAT', 'time_formatted', 'enmo_mg']

    # Initialize a flag to determine if the output file exists (for schema handling)
    first_file = True

    # Loop through all subdirectories in the source folder
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.parquet'):  # Check if it's a Parquet file
                try:
                    # Extract the part of the folder name after 'id='
                    folder_name = os.path.basename(root)
                    if "id=" in folder_name:
                        # Parse the id from the folder name
                        id_value = folder_name.split("id=")[1]
                        file_path = os.path.join(root, file)

                        # Read the Parquet file
                        print(f"Processing file: {file_path}")
                        df = pd.read_parquet(file_path)

                        # Log available columns
                        print(f"Columns in {file_path}: {df.columns.tolist()}")

                        # Ensure the DataFrame contains the necessary columns for transformation
                        # Add the 'id' column if it's not already present (derived from the folder name)
                        df['id'] = id_value

                        # Add the 'time_formatted' column if it needs to be created
                        if 'time_of_day' in df.columns:
                            df['time_formatted'] = pd.to_datetime(df['time_of_day'] / 1000000000, unit='s').dt.strftime('%H:%M:%S')
                        else:
                            df['time_formatted'] = pd.NA  # Create it as NaN if not available

                        # Add the 'enmo_mg' column if 'enmo' exists in the original file
                        if 'enmo' in df.columns:
                            df['enmo_mg'] = df['enmo'] * 1000
                        else:
                            df['enmo_mg'] = pd.NA  # Create it as NaN if 'enmo' is missing

                        # Filter out non-wear entries if 'non-wear_flag' exists
                        if 'non-wear_flag' in df.columns:
                            df = df[df['non-wear_flag'] == 0]

                        # Keep only necessary columns (add them if needed)
                        df = df[columns]

                        # Convert to a Parquet table for writing
                        table = pa.Table.from_pandas(df)

                        # Write to the Parquet file incrementally
                        if first_file:
                            pq.write_table(table, output_file)
                            first_file = False
                        else:
                            # Append to the Parquet file
                            with pq.ParquetWriter(output_file, table.schema, use_dictionary=True) as writer:
                                writer.write_table(table)

                except Exception as e:
                    print(f"Error processing file {file}: {e}")

    print(f"All files have been processed and combined into {output_file}")

In [ ]:
def modify_parquet_with_intensity(input_file, output_file):
    """
    Modifies a Parquet file by adding an 'intensity' column based on 'enmo_mg' values.
    
    Parameters:
        input_file (str): Path to the input Parquet file.
        output_file (str): Path to save the modified Parquet file.
    """
    # Step 1: Open the Parquet file and create a ParquetReader
    reader = pq.ParquetFile(input_file)

    # Step 2: Create an empty list to collect modified rows
    modified_rows = []

    # Iterate over row groups in the Parquet file
    for i in range(reader.num_row_groups):
        # Read each row group into a PyArrow Table
        table = reader.read_row_group(i)
        
        # Convert the table to a pandas DataFrame
        df = table.to_pandas()
        
        # Step 3: Add the conditional column based on 'enmo_mg'
        def categorize_intensity(row):
            if row['enmo_mg'] <= 50:
                return 'still'
            elif row['enmo_mg'] <= 200:
                return 'light'
            elif row['enmo_mg'] <= 500:
                return 'moderate'
            elif row['enmo_mg'] <= 750:
                return 'high'
            elif row['enmo_mg'] > 750:
                return 'vigorous'
            else:
                return None

        # Apply the function to each row to create the 'intensity' column
        df['intensity'] = df.apply(categorize_intensity, axis=1)
        
        # Convert the modified DataFrame back to a PyArrow Table and append it to the list
        modified_rows.append(pa.Table.from_pandas(df))

    # Step 4: Concatenate all modified tables into one large table
    final_table = pa.concat_tables(modified_rows)

    # Step 5: Write the modified table to a new Parquet file
    pq.write_table(final_table, output_file)

    print(f"Parquet file saved with the new column at {output_file}")

In [ ]:
def calculate_intensity_category(input_file, output_file):

    # Read the Parquet file
    df = pd.read_parquet(input_file)

    # Filter only relevant intensity levels
    relevant_intensities = ['still', 'light', 'moderate', 'high', 'vigorous']

    # Filter rows with relevant intensities
    filtered_df = df[df['intensity'].isin(relevant_intensities)].copy()

    # Calculate the number of rows for each intensity level per ID and date
    result = (
        filtered_df.groupby(['id', 'relative_date_PCIAT', 'intensity'])
        .size()
        .reset_index(name='count')
        .pivot(index=['id', 'relative_date_PCIAT'], columns='intensity', values='count')
        .fillna(0)
        .reset_index()
    )

    # Convert counts to minutes (5 seconds per row)
    result['still'] = result.get('still', 0) * 5 / 60
    result['light'] = result.get('light', 0) * 5 / 60
    result['moderate'] = result.get('moderate', 0) * 5 / 60
    result['high'] = result.get('high', 0) * 5 / 60
    result['vigorous'] = result.get('vigorous', 0) * 5 / 60

    # Rename columns to match required output format
    result.rename(columns={
        'still': 'total_time_still',
        'light': 'total_time_light',
        'moderate': 'total_time_moderate',
        'high': 'total_time_high',
        'vigorous': 'total_time_vigorous'
    }, inplace=True)

    # Ensure missing columns are included with zeros
    for col in [
        'total_time_still', 'total_time_light', 
        'total_time_moderate', 'total_time_high', 
        'total_time_vigorous'
    ]:
        if col not in result.columns:
            result[col] = 0

    # Save the result to CSV
    result.to_csv(output_file, index=False)


In [ ]:
def calculate_average_times(input_csv, output_csv):

    # Load the CSV file
    df = pd.read_csv(input_csv, delimiter=",", encoding="utf-8")
    print("Input CSV loaded successfully.")
    print(df.head())  # Preview the input data (optional)

    # Group by 'id' and calculate the mean for total_time columns across all dates
    averages_per_id = (
        df.groupby('id', as_index=False)
        [['total_time_still', 'total_time_light','total_time_moderate', 'total_time_high', 'total_time_vigorous']]
        .mean()
    )

    # Rename the columns to reflect average and unit
    averages_per_id.rename(columns={
        'total_time_still':'avg_time_still_(min/day)',
        'total_time_light':'avg_time_light_(min/day)',
        'total_time_moderate': 'avg_time_moderate_(min/day)',
        'total_time_high': 'avg_time_high_(min/day)',
        'total_time_vigorous': 'avg_time_vigorous_(min/day)'
    }, inplace=True)

    # Preview the result (optional)
    print("Averages calculated successfully.")
    print(averages_per_id.head())

    # Save the result to a new CSV file
    averages_per_id.to_csv(output_csv, index=False)
    print(f"Output CSV saved at {output_csv}")

In [ ]:
def merge_data_train(csv_file_path, parquet_combined_file_path, save_file_path):
    """
    Processes the data by reading, transforming, and saving it as described in the M language script.

    Parameters:
    csv_file_path (str): Path to the train/test CSV file.
    parquet_combined_file_path (str): Path to the average data CSV file.
    save_file_path (str): Path to save the processed data CSV file.
    """
    # Read the train and average CSV files
    train_df = pd.read_csv(csv_file_path)
    avg_df = pd.read_csv(parquet_combined_file_path)

    # Select relevant columns
    relevant_columns = [
        "id", "Basic_Demos-Age", "Basic_Demos-Sex", "CGAS-CGAS_Score",
        "Physical-BMI", "Physical-Waist_Circumference", "Physical-Diastolic_BP",
        "Physical-HeartRate", "Physical-Systolic_BP", "Fitness_Endurance-Time_Mins",
        "Fitness_Endurance-Time_Sec", "FGC-FGC_CU", "FGC-FGC_GSND", "FGC-FGC_GSD",
        "FGC-FGC_PU", "FGC-FGC_SRL", "FGC-FGC_SRR", "FGC-FGC_TL", "BIA-BIA_Activity_Level_num",
        "BIA-BIA_BMC", "BIA-BIA_BMR", "BIA-BIA_DEE", "BIA-BIA_ECW", "BIA-BIA_FFM",
        "BIA-BIA_FMI", "BIA-BIA_Fat", "BIA-BIA_ICW", "BIA-BIA_LDM", "BIA-BIA_LST",
        "BIA-BIA_SMM", "BIA-BIA_TBW", "SDS-SDS_Total_T", "PreInt_EduHx-computerinternet_hoursday",
        "sii"
    ]
    train_df = train_df[relevant_columns]

    # Add a new column for endurance time in seconds
    train_df["num_endure_sec"] = train_df["Fitness_Endurance-Time_Mins"] * 60 + train_df["Fitness_Endurance-Time_Sec"]

    columns_to_remove = ["Fitness_Endurance-Time_Mins", "Fitness_Endurance-Time_Sec"]
    train_df = train_df.drop(columns=columns_to_remove)

    # Rename columns
    column_renames = {
        "Basic_Demos-Age": "age", "Basic_Demos-Sex": "sex", "CGAS-CGAS_Score": "num_cgas",
        "Physical-BMI": "num_bmi", "SDS-SDS_Total_T": "num_sleep_loss",
        "PreInt_EduHx-computerinternet_hoursday": "cat_internet_use",
        "BIA-BIA_TBW": "num_tbm", "BIA-BIA_SMM": "num_smm", "BIA-BIA_LST": "num_lst",
        "BIA-BIA_LDM": "num_ldm", "BIA-BIA_ICW": "num_icw", "BIA-BIA_Fat": "num_fat",
        "BIA-BIA_FMI": "num_fmi", "BIA-BIA_FFM": "num_ffm", "BIA-BIA_ECW": "num_ecw",
        "BIA-BIA_DEE": "num_dee", "BIA-BIA_BMR": "num_bmr", "BIA-BIA_Activity_Level_num": "cat_activity_level",
        "BIA-BIA_BMC": "num_bmc", "FGC-FGC_TL": "num_tl", "FGC-FGC_SRR": "num_srr",
        "FGC-FGC_SRL": "num_srl", "FGC-FGC_PU": "num_pu", "FGC-FGC_GSD": "num_gsd",
        "FGC-FGC_GSND": "num_gsnd", "FGC-FGC_CU": "num_cu",
        "Physical-Waist_Circumference": "num_wc", "Physical-Diastolic_BP": "num_dbp",
        "Physical-Systolic_BP": "num_sbp", "Physical-HeartRate": "num_hr"
    }
    train_df.rename(columns=column_renames, inplace=True)

    # Merge with average data
    result_df = train_df.merge(avg_df, on="id", how="left")

    # Remove rows where 'sii' is null
    result_df = result_df.dropna(subset=["sii"])

    # Save the processed DataFrame
    result_df.to_csv(save_file_path, index=False)


In [ ]:
def merge_data_test(csv_file_path, parquet_combined_file_path, save_file_path):
    """
    Processes the data by reading, transforming, and saving it as described in the M language script.

    Parameters:
    csv_file_path (str): Path to the train/test CSV file.
    parquet_combined_file_path (str): Path to the average data CSV file.
    save_file_path (str): Path to save the processed data CSV file.
    """
    # Read the train and average CSV files
    test_df = pd.read_csv(csv_file_path)
    avg_df = pd.read_csv(parquet_combined_file_path)

    # Select relevant columns
    relevant_columns = [
        "id", "Basic_Demos-Age", "Basic_Demos-Sex", "CGAS-CGAS_Score",
        "Physical-BMI", "Physical-Waist_Circumference", "Physical-Diastolic_BP",
        "Physical-HeartRate", "Physical-Systolic_BP", "Fitness_Endurance-Time_Mins",
        "Fitness_Endurance-Time_Sec", "FGC-FGC_CU", "FGC-FGC_GSND", "FGC-FGC_GSD",
        "FGC-FGC_PU", "FGC-FGC_SRL", "FGC-FGC_SRR", "FGC-FGC_TL", "BIA-BIA_Activity_Level_num",
        "BIA-BIA_BMC", "BIA-BIA_BMR", "BIA-BIA_DEE", "BIA-BIA_ECW", "BIA-BIA_FFM",
        "BIA-BIA_FMI", "BIA-BIA_Fat", "BIA-BIA_ICW", "BIA-BIA_LDM", "BIA-BIA_LST",
        "BIA-BIA_SMM", "BIA-BIA_TBW", "SDS-SDS_Total_T", "PreInt_EduHx-computerinternet_hoursday"
    ]
    test_df = test_df[relevant_columns]

    # Add a new column for endurance time in seconds
    test_df["num_endure_sec"] = test_df["Fitness_Endurance-Time_Mins"] * 60 + test_df["Fitness_Endurance-Time_Sec"]

    columns_to_remove = ["Fitness_Endurance-Time_Mins", "Fitness_Endurance-Time_Sec"]
    test_df = test_df.drop(columns=columns_to_remove)

    # Rename columns
    column_renames = {
        "Basic_Demos-Age": "age", "Basic_Demos-Sex": "sex", "CGAS-CGAS_Score": "num_cgas",
        "Physical-BMI": "num_bmi", "SDS-SDS_Total_T": "num_sleep_loss",
        "PreInt_EduHx-computerinternet_hoursday": "cat_internet_use",
        "BIA-BIA_TBW": "num_tbm", "BIA-BIA_SMM": "num_smm", "BIA-BIA_LST": "num_lst",
        "BIA-BIA_LDM": "num_ldm", "BIA-BIA_ICW": "num_icw", "BIA-BIA_Fat": "num_fat",
        "BIA-BIA_FMI": "num_fmi", "BIA-BIA_FFM": "num_ffm", "BIA-BIA_ECW": "num_ecw",
        "BIA-BIA_DEE": "num_dee", "BIA-BIA_BMR": "num_bmr", "BIA-BIA_Activity_Level_num": "cat_activity_level",
        "BIA-BIA_BMC": "num_bmc", "FGC-FGC_TL": "num_tl", "FGC-FGC_SRR": "num_srr",
        "FGC-FGC_SRL": "num_srl", "FGC-FGC_PU": "num_pu", "FGC-FGC_GSD": "num_gsd",
        "FGC-FGC_GSND": "num_gsnd", "FGC-FGC_CU": "num_cu",
        "Physical-Waist_Circumference": "num_wc", "Physical-Diastolic_BP": "num_dbp",
        "Physical-Systolic_BP": "num_sbp", "Physical-HeartRate": "num_hr"
    }
    test_df.rename(columns=column_renames, inplace=True)

    # Merge with average data
    result_df = test_df.merge(avg_df, on="id", how="left")

    # Save the processed DataFrame
    result_df.to_csv(save_file_path, index=False)


In [ ]:
def impute_missing_values(input_csv_path, output_csv_path):
    """
    Reads a CSV file, fills missing values in numerical and categorical columns,
    and saves the processed data to another CSV file.

    Parameters:
    input_csv_path (str): Path to the input CSV file.
    output_csv_path (str): Path to save the processed CSV file.
    """
    # Read the CSV file
    df = pd.read_csv(input_csv_path)

    # Identify numerical and categorical columns based on prefixes
    numerical_columns = [col for col in df.columns if col.startswith('num_')]
    categorical_columns = [col for col in df.columns if col.startswith('cat_')]

    # Function to fill missing data
    def impute_data(df):
        # Impute numerical columns
        for col in numerical_columns:
            # Group by age and sex, then calculate the mean for each group
            # Use transform and fallback to overall mean if group data is missing
            df[col] = df.groupby(['age', 'sex'])[col].transform(
                lambda x: x.fillna(x.mean() if not x.isnull().all() else df[col].mean())
            )

        # Impute categorical columns
        for col in categorical_columns:
            # Group by age and sex, then calculate the mode for each group
            # Use transform and fallback to overall mode if group data is missing
            df[col] = df.groupby(['age', 'sex'])[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else df[col].mode()[0])
            )

        return df

    # Apply the imputation function
    df_imputed = impute_data(df)

    # Save the imputed data back to a CSV file
    df_imputed.to_csv(output_csv_path, index=False)

    print(f"Data has been imputed and saved to {output_csv_path}.")

In [ ]:
# os.path.join(working_dir, )

Change this part depend on where to run, maybe run on Kaggle or local

In [ ]:
# input_dir = r"E:\MachineLearning\inputs"
# working_dir = r"E:\MachineLearning\working"
input_dir = r"/kaggle/input/child-mind-institute-problematic-internet-use"
working_dir = r"/kaggle/working"

Process the train data

In [ ]:
parquet_src_train = r'series_train.parquet'
combined_train = r'combined_train.parquet'
intensity_per_day_train = r'intensity_pd_train.csv'
avg_intensity_pd_train = r'avg_int_pd_train.csv'
train_csv = r'train.csv'
merged_train_data = r'merged_train.csv'
imputed_train_data = r'imputed_train.csv'

In [ ]:
combine_parquet(os.path.join(input_dir, parquet_src_train), os.path.join(working_dir, combined_train))
modify_parquet_with_intensity(os.path.join(working_dir, combined_train),os.path.join(working_dir, combined_train))
calculate_intensity_category(os.path.join(working_dir, combined_train),os.path.join(working_dir, intensity_per_day_train))
calculate_average_times(os.path.join(working_dir, intensity_per_day_train),os.path.join(working_dir, avg_intensity_pd_train))
merge_data_train(os.path.join(input_dir, train_csv), os.path.join(working_dir, avg_intensity_pd_train), os.path.join(working_dir, merged_train_data))
# impute_missing_values(os.path.join(working_dir, merged_train_data), os.path.join(working_dir, imputed_train_data))

Process the test data

In [ ]:
parquet_src_test = r'series_test.parquet'
combined_test = r'combined_test.parquet'
intensity_per_day_test = r'intensity_pd_test.csv'
avg_intensity_pd_test = r'avg_int_pd_test.csv'
test_csv = r'test.csv'
merged_test_data = r'merged_test.csv'
imputed_test_data = r'imputed_test.csv'

In [ ]:
combine_parquet(os.path.join(input_dir, parquet_src_test), os.path.join(working_dir, combined_test))
modify_parquet_with_intensity(os.path.join(working_dir, combined_test),os.path.join(working_dir, combined_test))
calculate_intensity_category(os.path.join(working_dir, combined_test),os.path.join(working_dir, intensity_per_day_test))
calculate_average_times(os.path.join(working_dir, intensity_per_day_test),os.path.join(working_dir, avg_intensity_pd_test))
merge_data_test(os.path.join(input_dir, test_csv), os.path.join(working_dir, avg_intensity_pd_test), os.path.join(working_dir, merged_test_data))
# impute_missing_values(os.path.join(working_dir, merged_test_data), os.path.join(working_dir, imputed_test_data))

ML part start here

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import shutil
import pandas as pd
import pyarrow.parquet as pq
from pyarrow import fs
import pyarrow as pa

In [ ]:
# working_dir = r"E:\MachineLearning\working"
working_dir = r"/kaggle/working"
train_data_path = r'merged_train.csv'
test_data_path = r'merged_test.csv'

In [ ]:
# Load data from CSV
train_data = pd.read_csv(os.path.join(working_dir, train_data_path))  
test_data = pd.read_csv(os.path.join(working_dir, test_data_path))  

In [ ]:
columns_to_convert_train = ["cat_internet_use", "cat_activity_level", "sii"]
for col in columns_to_convert_train:
    train_data[col] = train_data[col].astype(int)

columns_to_convert_test = ["cat_internet_use", "cat_activity_level"]
for col in columns_to_convert_test:
    test_data[col] = test_data[col].astype(int)

In [ ]:
train_data = train_data.drop_duplicates(subset='id', keep='first')
test_data = test_data.drop_duplicates(subset='id', keep='first')

In [ ]:
# Split the train data into X (features) and y (target)
X_train = train_data.drop(columns=['id','sii'])  # Features (exclude target column)
y_train = train_data['sii']  # Target variable (0-3 categories)

# Split the test data into X (features) and y (target)
X_test = test_data.drop(columns=['id'])  # Features (exclude target column)

In [ ]:
import lightgbm as lgb
# Train the Random Forest model (here I picked number 69 hehe)
model = lgb.LGBMClassifier(n_estimators = 500, max_depth = 10, num_leaves= 50, learning_rate= 0.01, class_weight= 'balanced')
model.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_pred_test = model.predict(X_test)
print(y_pred_test)

In [ ]:
# Assign predictions to the test_data DataFrame
test_data['pred_sii'] = y_pred_test

# Select only the required columns for the output
output_df = test_data[['id', 'pred_sii']].copy()

# Rename the column to 'sii'
output_df.rename(columns={'pred_sii': 'sii'}, inplace=True)

# Sort by the 'id' column
output_df.sort_values(by='id', inplace=True)

# Save the DataFrame to a CSV file
output_df.to_csv('/kaggle/working/submission.csv', index=False, encoding='utf-8')